# 【課題】高エネルギー実験で生成された荷電粒子の飛跡を見つける

高エネルギー物理の実験では必須の技術である「**荷電粒子飛跡の再構成**」を、変分量子固有値ソルバー法を使って実現する。

まず、必要なライブラリを最初にインポートします。

In [4]:
# Tested with python 3.7.9, qiskit 0.23.5, numpy 1.20.1、hepqpr-qallse 0.1.0
import numpy as np
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer
from qiskit.circuit.library import TwoLocal
from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver, NumPyEigensolver
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.aqua.components.optimizers import SPSA, COBYLA
from qiskit.aqua import QuantumInstance

## ハミルトニアンの構成とVQEの実行

課題として考えるのは、**Trackingを実現するためのハミルトニアンを構成し、それをVQEに実装して実行する**ことです。

### 問題

以上のデータから、VQEで用いるハミルトニアンを構成してみてください。

**ヒント1**：この形式のデータを以下のコードを使って読み込むとします。

In [5]:
file_r = 'data/QUBO_05pct_input.txt'
from ast import literal_eval
with open(file_r, "r") as f:
    line = f.read()
    Q = literal_eval(line)
print("Q size =",len(Q))


n_max = 100

nvar = 0
key_i = []
b_ij = np.zeros((n_max,n_max))
for (k1, k2), v in Q.items():
    if k1 == k2:
        b_ij[nvar][nvar] = v
        key_i.append(k1)
        nvar += 1

for (k1, k2), v in Q.items():
    if k1 != k2:
        for i in range(nvar):
            for j in range(nvar):
                if k1 == key_i[i] and k2 == key_i[j]:
                    if i < j:
                        b_ij[i][j] = v
                    else:
                        b_ij[j][i] = v

b_ij = b_ij[:nvar,:nvar]
print("# of segments =",nvar)

Q size = 46
# of segments = 10


セグメント数を$N$（上のコードではnvar）とすると、この時b_ijは$N$行$N$列の正方行列になります。実はこのデータは、**QUBO**（*Quadratic Unconstrained Binary Optimization*、2次制約無し2値最適化）と呼ばれる問題として解くことができる形式で与えられています。QUBOは量子ビット$T$がバイナリー値（0か1）を持つ場合に、以下の式で与えられる目的関数$O$を最小化する問題として定義されます。

$$
O(b,T) = \sum_{i=1}^Nb_{ii}T_i + \sum_{i=1}^N\sum_{j=1\:(i<j)}^Nb_{ij}T_iT_j
$$

$T^2=T$でもあるため、上式は簡単に

$$
O(b,T) = \sum_{i=1}^N\sum_{j=1}^Nb_{ij}T_iT_j
$$

と書くこともできます。$T$は{0,1}のバイナリー値を持ちますが、シンプルな計算で{+1,-1}を持つ量子ビットに変換することができます。{+1,-1}はパウリ$Z$演算子の固有値でもあるため、パウリ$Z$演算子を使って目的関数$O$をハミルトニアンとして書くことができれば、そのまま量子回路に実装することができます。

以下のスペースに、どのような変換が可能か等を含め、VQEでTrackingを実行するために必要な量子演算を定義してください。

**ヒント2**：まず、{0,1}のバイナリー値を持つ$T$を{+1,-1}をもつスピン$s$に変換します。$T=0$を$s=1$、$T=1$を$s=-1$に対応させるとします。この関係の下で、目的関数$O$を

$$
H(h,J,s) = \sum_{i=1}^Nh_is_i + \sum_{i=1}^N\sum_{j=1\:(i<j)}^NJ_{ij}s_is_j
$$

となるような関数$H$に書き換えてみてください。この関数$H$はイジング模型のハミルトニアンと同じ形になっています。

In [6]:
from qiskit.quantum_info import Pauli
from qiskit.aqua.operators import PrimitiveOp

##################
### EDIT BELOW ###
##################

# ステップ１：{0,1}を取るTに対して定義されているb_ijを、{+1,-1}を取る変数sに対して定義しなおす。
J_ij = np.zeros((nvar,nvar))
for i in range(nvar):
    for j in range(nvar):
        if i >= j:
            continue        
        J_ij[i][j] = b_ij[i][j]+b_ij[j][i]
        if J_ij[i][j] == 0:
            continue
        
h_i = np.zeros(nvar)
for i in range(nvar):
    bias = 0
    for k in range(nvar):
        bias += b_ij[i][k]+b_ij[k][i]
    bias *= -1
    h_i[i] = bias
    if h_i[i] == 0:
        continue

# ステップ２：変数sをパウリZ演算子を使って実装する。

#def get_qubitops(...):
#
#
#    return ??
def get_qubitops(J_ij,h_i,penalty=1e6):
    num_qubits = len(h_i)
    print("Number of qubits (selected triplets) =",num_qubits)
    weighted_sum = 0
    
    zero = np.zeros(num_qubits, dtype=bool)
    for i in range(num_qubits):
        for j in range(num_qubits):
            if i >= j:
                continue
            if J_ij[i][j] == 0:
                continue
            print("J_ij[",i,"][",j,"]=",J_ij[i][j])
            vp = np.zeros(num_qubits, dtype=bool)
            vp[i] = True
            vp[j] = True
            weighted_sum += J_ij[i][j]*PrimitiveOp(Pauli(vp, zero))

    zero = np.zeros(num_qubits, dtype=bool)
    for i in range(num_qubits):
        print("h_i[",i,"]=",h_i[i])
        vp = np.zeros(num_qubits, dtype=bool)
        vp[i] = True
        weighted_sum += h_i[i]*PrimitiveOp(Pauli(vp, zero))

    return weighted_sum

##################
### EDIT ABOVE ###
##################

このコードの部分を課題として提出してください。

**ヒント3**：get_qubitopsは、パウリ$Z$演算子を使って実装した観測量$H$を返す関数です。Qiskitでパウリ$Z$演算子とそのテンソル積を実装するには、qiskit.quantum_info.Pauliクラス（[ここ](https://qiskit.org/documentation/stubs/qiskit.quantum_info.Pauli.html)を参照）とqiskit.aqua.operatorsライブラリ（[ここ](https://qiskit.org/documentation/apidoc/qiskit.aqua.operators.html)を参照）を使うのが便利です。セグメント間の相互作用の強さを表す$J_{ij}$は、2つのパウリ$Z$演算子のテンソル積に対する係数として導入する必要があります。それをqiskit.quantum_info.Pauliを使ってどのように書くでしょうか？$h_i$は単一パウリ$Z$演算子の係数になります。そして、最終的に測定する観測量$H$は、それらパウリ$Z$演算子の線形結合になりますね。

### VQEによる近似解の探索

上で定義したハミルトニアンを元に、VQEを使ってエネルギーの最小固有値（の近似解）を求めていきます。ただその前に、このハミルトニアンの行列を対角化して、エネルギーの最小固有値とその固有ベクトルを厳密に計算した場合の答えを出してみましょう。

In [7]:
# ハミルトニアンオペレータを取得
##################
### EDIT BELOW ###
##################
qubitOp = get_qubitops(J_ij,h_i)
##################
### EDIT ABOVE ###
##################
print("")
print("total number of qubits = ",qubitOp.num_qubits)

# ハミルトニアン行列を対角化して、エネルギーの最小固有値と固有ベクトルを求める
ee  = NumPyMinimumEigensolver(qubitOp)
result = ee.run()

# 最小エネルギーに対応する量子ビットの組み合わせを表示
print('Eigensolver: objective =', result.eigenvalue.real)
x = sample_most_likely(result.eigenstate)
print('Eigensolver: x =',x)

samples_eigen = {}
for i in range(nvar):
    samples_eigen[key_i[i]] = x[i]

Number of qubits (selected triplets) = 10
J_ij[ 0 ][ 1 ]= 1.0
J_ij[ 0 ][ 3 ]= 1.0
J_ij[ 0 ][ 4 ]= 1.0
J_ij[ 0 ][ 5 ]= 1.0
J_ij[ 0 ][ 6 ]= -0.2422171861077038
J_ij[ 0 ][ 8 ]= 1.0
J_ij[ 0 ][ 9 ]= 1.0
J_ij[ 1 ][ 2 ]= -0.20194633024258613
J_ij[ 1 ][ 3 ]= 1.0
J_ij[ 1 ][ 4 ]= 1.0
J_ij[ 1 ][ 5 ]= 1.0
J_ij[ 1 ][ 6 ]= 1.0
J_ij[ 1 ][ 7 ]= 1.0
J_ij[ 1 ][ 8 ]= 1.0
J_ij[ 1 ][ 9 ]= -0.24622296145101896
J_ij[ 2 ][ 4 ]= 1.0
J_ij[ 2 ][ 5 ]= 1.0
J_ij[ 2 ][ 6 ]= -0.8163519177517079
J_ij[ 2 ][ 7 ]= 1.0
J_ij[ 3 ][ 4 ]= 1.0
J_ij[ 3 ][ 5 ]= 1.0
J_ij[ 3 ][ 6 ]= -0.9622918217115106
J_ij[ 3 ][ 7 ]= -0.22174336478384057
J_ij[ 3 ][ 8 ]= -0.9698465871822172
J_ij[ 3 ][ 9 ]= 1.0
J_ij[ 4 ][ 5 ]= -0.2093159882343949
J_ij[ 4 ][ 6 ]= 1.0
J_ij[ 4 ][ 7 ]= 1.0
J_ij[ 4 ][ 8 ]= -0.24685399312094367
J_ij[ 4 ][ 9 ]= 1.0
J_ij[ 5 ][ 6 ]= 1.0
J_ij[ 5 ][ 7 ]= 1.0
J_ij[ 6 ][ 7 ]= 1.0
J_ij[ 6 ][ 9 ]= 1.0
J_ij[ 7 ][ 9 ]= 1.0
J_ij[ 8 ][ 9 ]= 1.0
h_i[ 0 ]= -5.780035929747845
h_i[ 1 ]= -6.554566686862729
h_i[ 2 ]= -2.5507675951639466
h_

次に、同じハミルトニアンモデルをVQEに実装して、最小エネルギーを求めてみます。オプティマイザーとしてSPSAあるいはCOBYLAを使う場合のコードは以下のようになります。

まず最初に、VQE用の量子回路を作ります。

In [8]:
# VQE用の回路を作る：ここではTwoLocalという組み込み関数を使う
seed = 10598
spsa = SPSA(maxiter=300)
cobyla = COBYLA(maxiter=500)
two = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', 'linear', reps=1)
print(two)

     ┌──────────┐   ┌───────────┐                                       »
q_0: ┤ RY(θ[0]) ├─■─┤ RY(θ[10]) ├───────────────────────────────────────»
     ├──────────┤ │ └───────────┘┌───────────┐                          »
q_1: ┤ RY(θ[1]) ├─■───────■──────┤ RY(θ[11]) ├──────────────────────────»
     ├──────────┤         │      └───────────┘┌───────────┐             »
q_2: ┤ RY(θ[2]) ├─────────■────────────■──────┤ RY(θ[12]) ├─────────────»
     ├──────────┤                      │      └───────────┘┌───────────┐»
q_3: ┤ RY(θ[3]) ├──────────────────────■────────────■──────┤ RY(θ[13]) ├»
     ├──────────┤                                   │      └───────────┘»
q_4: ┤ RY(θ[4]) ├───────────────────────────────────■────────────■──────»
     ├──────────┤                                                │      »
q_5: ┤ RY(θ[5]) ├────────────────────────────────────────────────■──────»
     ├──────────┤                                                       »
q_6: ┤ RY(θ[6]) ├─────────────────────

バックエンドしてqasm_simulatorを使い、実行した結果を書き出します。

In [9]:
# VQEの実行
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend=backend, shots=1024, seed_simulator=seed)
vqe = VQE(qubitOp, two, spsa)
#vqe = VQE(qubitOp, two, cobyla)
result = vqe.run(quantum_instance)

# 最小エネルギーに対応する量子ビットの組み合わせを表示
print('')
print('VQE: objective =', result.eigenvalue.real)
x = sample_most_likely(result.eigenstate)
print('VQE x =',x)

samples_vqe = {}
for i in range(nvar):
    samples_vqe[key_i[i]] = x[i]


VQE: objective = -28.50227393072697
VQE x = [0 0 0 1 0 0 0 0 1 0]


### おまけ

Trackingがうまく行っても、この答えだと0と1が並んでいるだけで面白くないですよね。正しく飛跡が見つかったかどうか目で確認するため、以下のコードを走らせてみましょう。

このコードは、QUBOを定義する時に使った検出器のヒット位置をビーム軸に垂直な平面でプロットして、どのヒットが選ばれたかを分かりやすく可視化したものです。緑の線が実際に見つかった飛跡で、青の線を含めたものが全体の飛跡の候補です。この実習では限られた数の量子ビットしか使っていないため、大部分の飛跡は見つけられていませんが、緑の線から計算に使った3点ヒットからは正しく飛跡が見つかっていることが分かると思います。

In [ ]:
from hepqpr.qallse import *
input_path = './data/event000001000-hits.csv'
dw = DataWrapper.from_path(input_path)

# get the results
#all_doublets = Qallse.process_sample(samples_eigen)
all_doublets = Qallse.process_sample(samples_vqe)

final_tracks, final_doublets = TrackRecreaterD().process_results(all_doublets)
#print("all_doublets =",all_doublets)
#print("final_tracks =",final_tracks)
#print("final_doublets =",final_doublets)

p, r, ms = dw.compute_score(final_doublets)
trackml_score = dw.compute_trackml_score(final_tracks)

print(f'SCORE  -- precision (%): {p * 100}, recall (%): {r * 100}, missing: {len(ms)}')
print(f'          tracks found: {len(final_tracks)}, trackml score (%): {trackml_score * 100}')

from hepqpr.qallse.plotting import iplot_results, iplot_results_tracks
dims = ['x', 'y']
_, missings, _ = diff_rows(final_doublets, dw.get_real_doublets())
dout = 'plot-ising_found_tracks.html'
iplot_results(dw, final_doublets, missings, dims=dims, filename=dout)

**提出するもの**
- ハミルトニアンを実装する部分のコード